# Заготовка для работы Кластерный анализ

импортируем библиотеки

In [ ]:
%pylab inline
from ipywidgets import interact

import seaborn as sns

from scipy.cluster.hierarchy import dendrogram, linkage, optimal_leaf_ordering, cut_tree

## 1. Загрузка данных

> *опишите источник данных. информация по скольким объектам содержится в файле?*

In [ ]:
u='spectral_data.txt'  # проверьте наличие файла в рабочей папке
X = loadtxt(u)
X.shape  #форма массива со значениями прочитанного файла

> *Ниже задаются имена в виде порядковых номеров. Если Вы как-то различаете объекты, информацию о которых загрузили, то раскомментируйте вторую строку и задайте имена по количеству строк в массиве, они будут использоваться для подписей. Обоснуйте названия.*

In [ ]:
names = arange(X.shape[0])+1
# names = ['pi','pi','iu','iu','iu','iu','iu','i']
names

## 2. Подбор параметров связи

>_попробуйте несколько типов связей . копируйте код и изменяйте параметры method= (2-) и  metric= в функции `linkage`._

> по-умолчанию связи рассчитываются с параметрами `linkage(y, method='single', metric='euclidean')`

> варианты для method:
- 'single' - Алгоритм ближайшей точки
- 'complete' - Алгоритм удаленной точки
- 'average' - Среднее расстояние со всеми соседями
- 'centroid' - центроид - центр облака, занимаемого группой
- 'median' - медиана дистанций до соседей
- 'ward' - минимизация внутрикластерной дисперсии по методу Уарда

> варианты для metric:
- 'euclidean' - Евклида, прямое расстояние
- 'cosine' - косинусное расстояние - угол между векторами от центра
- 'cityblock' - квартальное расстояние (если поворачивать только на прямые углы)
- 'correlation' - корреляция
- 'hamming' - Хэмминга, доля совпадающих значений
- 'jaccard' - Жаккара, отношение количества различий к совпадениям
- 'mahalanobis' - Махаланобиса, дистанция до центра группы, нормализованная к ее дисперсии

> Примеры эффективности разных методов кластеризации по двум признакам (в двумерном пространстве).
![](plot_linkage_comparison.png)

In [ ]:
dendrogram( linkage(X, 'single', "euclidean"), labels=names);

In [ ]:
dendrogram( linkage(X, 'average', 'euclidean'), labels=names);

In [ ]:
dendrogram( linkage(X, 'median', 'euclidean'), labels=names);

In [ ]:
dendrogram( linkage(X, 'complete', 'euclidean'), labels=names);

In [ ]:
dendrogram( linkage(X, 'ward', "euclidean"), labels=names);

In [ ]:
dendrogram( linkage(X, 'average', 'cosine'), labels=names);

In [ ]:
dendrogram( linkage(X, 'single', 'cityblock'), labels=names);

In [ ]:
dendrogram( linkage(X, 'complete', 'cityblock'), labels=names);

In [ ]:
dendrogram( linkage(X, 'single', 'correlation'), labels=names);

In [ ]:
dendrogram( linkage(X, 'single', 'correlation', True), labels=names);

In [ ]:
dendrogram( linkage(X, 'single', 'hamming'), labels=names);

In [ ]:
# dendrogram( linkage(X, '...', '...'), labels=names);

>_Оставьте наиболее понравившиеся Вам деревья. Поясните, какие значения параметров способствуют разделению на кластеры, сходному с интуитивным. Обоснуйте выбор победителя.
Оформите окончательный выбор в следующем рисунке._

In [ ]:
method='single'
metric='euclidean'
lm = linkage(X, method, metric, True)
porog = 0.75 * lm[:,2].max() # задайте значение для пунктирной линии

figure(figsize=(8, 3))
horizont = 0
height = 1
ax = axes((0,horizont,.8,height))
dendrogram(lm, color_threshold=porog, labels=names);
axhline(porog, ls='--', color='gray')
ylabel(metric +' distance')

axes((.8,horizont,.12,height))
sns.kdeplot(lm[:,2], bw=0.04*lm[:,2].max(), vertical=True)
ylim(ax.get_ylim())
axis('off')

> _подберите положение порога (пунктирной линии), чтобы "разрезать" дерево на крупные, но компактные ветви._

## 2. Цветная разметка кластеров

"Срежем" дерево на заданном уровне.

In [ ]:
cl = cut_tree(lm, height=porog)
cl

Для каждого объекта мы получили индекс кластера (начиная с 0), к которому он принадлежит согласно выбранным параметрам.

> _Загрузите несколько переменных (og, fs, iiab), которые Вы сохранили в предыдущей работе. Постройте заключительный рисунок._

In [ ]:
o = load('для рисунка.npz')
o.files

In [ ]:
og = o['og']
fs = o['fs']
iiab = o['iiab']
iiab

In [ ]:
coco = cm.tab10(np.unique(cl))
figure(figsize=(12, 4))
amax = max(og)                  # рассчитайте максимум для высоты полос
t = arange(len(og))/fs
for i in arange(len(cl)):
    vlines(t[iiab[i]], 0, amax, color=coco[cl[i]], lw=10, alpha=.8, label=names[i])
plot(t, og, color='k', alpha=.8);
xlabel('Время, с')
legend();

>_Оцените адекватность выделения кластеров на временной развертке. Соответствует ли сходству и различиям звуков на слух?_

> # _Критерии оценки_
> Критерии оценивания за расчетно-графическую работу (баллов):
- работа сдана в правильном формате +30
- работа сдана в форме реферата +10
- название не по шаблону -1
- не убраны инструкции и комментарии в конце строчек кода -5
- не везде комментарии, нет выводов -5
- не обоснован выбор параметров  -5
- источник данных не соответствует требованиям -5
- некорректное суждение при ответе на вопрос -2 за каждое 